In [75]:

#Script to obtain data 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

#Libraries to create the multiclass model

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
#Import tensorflow and disable the v2 behavior and eager mode


import tensorflow as tf
# tf.compat.v1.disable_eager_execution()
# tf.compat.v1.disable_v2_behavior()

#Library to validate the model
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score


In [76]:
df = pd.read_csv('./Data/Training/Train.csv')
df

,name,uri,genres,mood,length,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,timeSignature
0,Ensnared,1p5jbuZMzTZUXOlmI8VRIl,"['chillhop', 'indie jazz', 'indie soul', 'jazz...",calm,0 days 00:03:31.150000,0.81300,0.547,0.376,0.868000,0.1550,-9.531,0.0412,140.674,0.4660,4
1,Aspen,0w7HY9VhJF5KHPqhOyY6Sg,['lo-fi beats'],calm,0 days 00:01:54.545000,0.08440,0.828,0.412,0.894000,0.1260,-14.672,0.1280,110.054,0.1900,4
2,st. mark’s place,4HXuZde89plw90HUI8qGPN,[],calm,0 days 00:02:02.898000,0.90400,0.561,0.283,0.927000,0.1210,-15.499,0.1090,176.006,0.3320,4
3,Hawaii,6N0wKoHDvZfrylcBlKV9ZP,[],calm,0 days 00:02:51,0.89000,0.816,0.340,0.894000,0.2660,-9.118,0.2240,79.979,0.1500,4
4,Plane crash in Paradise,1F5nK1fVsd1aydJDNQ86lH,['lo-fi jazzhop'],calm,0 days 00:02:07.618000,0.32100,0.551,0.468,0.452000,0.2880,-11.611,0.4100,76.247,0.3750,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1772,Who (feat. BTS),2qG81jL9UIP54uS8gYyP4k,"['electropop', 'pop', 'post-teen pop']",sad,0 days 00:03:00.413000,0.55100,0.632,0.345,0.000000,0.1040,-8.000,0.0349,141.641,0.0818,3
1773,Fine Line,6VzcQuzTNTMFnJ6rBSaLH9,"['pop', 'post-teen pop']",sad,0 days 00:06:17.960000,0.17200,0.306,0.347,0.000130,0.0485,-8.500,0.0334,120.996,0.0511,4
1774,Won't Go Home Without You,6WEBwvsmpaoGxka0tSh5a7,"['pop', 'pop rock']",sad,0 days 00:03:51.173000,0.01490,0.737,0.615,0.000000,0.1020,-3.760,0.0317,110.023,0.4100,4
1775,Set Fire to the Rain,5PKWUDfQFtc5qqo8cs1gQp,"['british soul', 'pop', 'pop soul', 'uk pop']",sad,0 days 00:04:02.974000,0.00408,0.603,0.670,0.000002,0.1120,-3.882,0.0249,107.995,0.4450,4


In [77]:
col_features = df.columns[6:-3]
X= MinMaxScaler().fit_transform(df[col_features].dropna())
X2 = np.array(df[col_features])
Y = df['mood']

In [78]:
X

array([[5.66839378e-01, 3.70888158e-01, 8.90256410e-01, 1.47200179e-01,
        6.28352164e-01, 5.06142506e-02],
       [8.58031088e-01, 4.07894737e-01, 9.16923077e-01, 1.14787079e-01,
        4.09166489e-01, 1.57248157e-01],
       [5.81347150e-01, 2.75287829e-01, 9.50769231e-01, 1.09198614e-01,
        3.73907482e-01, 1.33906634e-01],
       ...,
       [7.63730570e-01, 6.16570724e-01, 0.00000000e+00, 8.79624455e-02,
        8.74397783e-01, 3.89434889e-02],
       [6.24870466e-01, 6.73108553e-01, 1.70256410e-06, 9.91393763e-02,
        8.69196333e-01, 3.05896806e-02],
       [9.09844560e-01, 6.42269737e-01, 0.00000000e+00, 8.36034425e-02,
        7.94201663e-01, 7.26044226e-02]])

In [79]:
col_features

Index(['danceability', 'energy', 'instrumentalness', 'liveness', 'loudness',
       'speechiness'],
      dtype='object')

In [80]:
#Encodethe categories
encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)


#Convert to  dummy (Not necessary in my case)
dummy_y = np_utils.to_categorical(encoded_y)

X_train,X_test,Y_train,Y_test = train_test_split(X,encoded_y,test_size=0.2,random_state=15)

target = pd.DataFrame({'mood':df['mood'].tolist(),'encode':encoded_y}).drop_duplicates().sort_values(['encode'],ascending=True)
target

,mood,encode
0,calm,0
1107,energetic,1
767,happy,2
1425,sad,3


In [81]:
def base_model():
    #Create the model
    model = Sequential()
    #Add 1 layer with 8 nodes,input of 4 dim with relu function
    model.add(Dense(8,input_dim=6,activation='relu'))
    #Add 1 layer with output 3 and softmax function
    model.add(Dense(4,activation='softmax'))
    #Compile the model using sigmoid loss function and adam optim
    opt = keras.optimizers.Adam(learning_rate=0.001)
    opt.learning_rate = .001
    model.compile(loss='categorical_crossentropy',optimizer=opt,
                 metrics=['accuracy'])
    return model

In [82]:
estimator = KerasClassifier(build_fn=base_model,epochs=300,batch_size=200,verbose=0)


In [83]:
encoded_y

array([0, 0, 0, ..., 3, 3, 3])

In [84]:
X

array([[5.66839378e-01, 3.70888158e-01, 8.90256410e-01, 1.47200179e-01,
        6.28352164e-01, 5.06142506e-02],
       [8.58031088e-01, 4.07894737e-01, 9.16923077e-01, 1.14787079e-01,
        4.09166489e-01, 1.57248157e-01],
       [5.81347150e-01, 2.75287829e-01, 9.50769231e-01, 1.09198614e-01,
        3.73907482e-01, 1.33906634e-01],
       ...,
       [7.63730570e-01, 6.16570724e-01, 0.00000000e+00, 8.79624455e-02,
        8.74397783e-01, 3.89434889e-02],
       [6.24870466e-01, 6.73108553e-01, 1.70256410e-06, 9.91393763e-02,
        8.69196333e-01, 3.05896806e-02],
       [9.09844560e-01, 6.42269737e-01, 0.00000000e+00, 8.36034425e-02,
        7.94201663e-01, 7.26044226e-02]])

In [85]:
#Evaluate the model using KFold cross validation
kfold = KFold(n_splits=10,shuffle=True)
results = cross_val_score(estimator,X_train, Y_train,cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100,results.std()*100))

Baseline: nan% (nan%)


In [86]:
estimator.fit(X_train,Y_train)
y_preds = estimator.predict(X_test)

FailedPreconditionError: Could not find variable training_132/Adam/beta_2. This could mean that the variable has been deleted. In TF1, it can also mean the variable is uninitialized. Debug info: container=localhost, status=Not found: Resource localhost/training_132/Adam/beta_2/N10tensorflow3VarE does not exist.
	 [[{{node training_132/Adam/Identity_2/ReadVariableOp}}]]

In [ ]:
results